In [1]:
# whether to log each feature and sequence status
verbose = True

In [2]:
import gc
import os
import pandas as pd
import numpy as np
import json
import datetime
import matplotlib.pyplot as plt
import itertools
import sys
from shutil import rmtree
sys.path.append('..')

In [3]:
from keras.models import load_model

Using TensorFlow backend.


In [4]:
# setup paths
pwd = os.getcwd().replace("notebooks","")
path_cache = pwd + 'cache/'
path_data = pwd + 'data/'

In [5]:
# setup logging
# any explicit log messages or uncaught errors to stdout and file /logs.log
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s",
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs")),
        logging.StreamHandler()
    ])
# init logger
logger = logging.getLogger()
# make logger aware of any uncaught exceptions
def handle_exception(exc_type, exc_value, exc_traceback):
    if issubclass(exc_type, KeyboardInterrupt):
        sys.__excepthook__(exc_type, exc_value, exc_traceback)
        return

    logger.error("Uncaught exception", exc_info=(exc_type, exc_value, exc_traceback))
sys.excepthook = handle_exception

In [6]:
from deepvideoclassification.architectures import Architecture

In [7]:
from deepvideoclassification.data import Data

# Load model weights from file 

> e.g. /models/123/model_best.h5

In [8]:
experiment = {
             'architecture': 'video_lrcnn_frozen',
             'dropout': 0.2,
             'layer_1_size': 256,
             'layer_2_size': 512,
             'layer_3_size': 256,
             'model_id': 1,
             'pooling': 'max',
             'pretrained_model_name': 'vgg16',
             'sequence_length': 20,
             'sequence_model': "LSTM",
             'sequence_model_layers': 2}

In [9]:
print(str(experiment["model_id"]) + "   " + "X"*60)
print(experiment)

architecture = Architecture(model_id = experiment['model_id'], 
                            architecture = experiment['architecture'], 
                            sequence_length = experiment['sequence_length'], 
                            pretrained_model_name = experiment['pretrained_model_name'],
                            pooling = experiment['pooling'],
                            sequence_model = experiment['sequence_model'],
                            sequence_model_layers = experiment['sequence_model_layers'],
                            layer_1_size = experiment['layer_1_size'],
                            layer_2_size = experiment['layer_2_size'],
                            layer_3_size = experiment['layer_3_size'],
                            dropout = experiment['dropout'],
                            verbose=True)

2020-04-14 06:52:10,790 [MainThread  ] [INFO ]  Loading data


1   XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
{'layer_3_size': 256, 'model_id': 1, 'dropout': 0.2, 'sequence_model_layers': 2, 'sequence_length': 20, 'layer_2_size': 512, 'layer_1_size': 256, 'pretrained_model_name': 'vgg16', 'sequence_model': 'LSTM', 'architecture': 'video_lrcnn_frozen', 'pooling': 'max'}


2020-04-14 06:52:11,283 [MainThread  ] [INFO ]  Features already cached: /mnt/seals/cache/features/vgg16/max/
2020-04-14 06:52:11,284 [MainThread  ] [INFO ]  Loading features sequence data into memory [may take a few minutes]


Done initializing data with #samples: train=10034, valid=1285, test=265


In [10]:
print(architecture.path_model + "model_best.h5")

/mnt/seals/models/1/model_best.h5


In [11]:
# load model weights
architecture.model = load_model(architecture.path_model + "model_best.h5")

# Load data and predict on test frames

In [12]:
sequence_length = experiment['sequence_length']
pretrained_model_name = experiment['pretrained_model_name']
pooling = experiment['pooling']

In [13]:
# build feature cache if it doesn't already exist
data = Data(sequence_length=sequence_length, 
            return_CNN_features=True,
            pretrained_model_name = pretrained_model_name,
            pooling=pooling)

2020-04-14 06:52:34,781 [MainThread  ] [INFO ]  Features already cached: /mnt/seals/cache/features/vgg16/max/
2020-04-14 06:52:34,782 [MainThread  ] [INFO ]  Loading features sequence data into memory [may take a few minutes]


Done initializing data with #samples: train=10034, valid=1285, test=265


In [14]:
labels = data.labels

In [49]:
# get class names from data object
class_names = []
for k in sorted(data.label_map.keys()):
    class_names.append(data.label_map[k])
class_names = [c.replace("label_","") for c in class_names]
class_names

['noseal', 'seal']

In [50]:
# get list of videos
videos = list(labels['video'].unique())

In [51]:
def get_features_path(video):
    return pwd + 'cache/features/' + experiment['pretrained_model_name'] + '/' + experiment['pooling'] + '/' + video + '.npy'
print(get_features_path(videos[0]))

/mnt/seals/cache/features/vgg16/max/s1-218.npy


In [ ]:
# collect predictions for each video
y_preds = []

for c, video in enumerate(videos):
    print("Computing frame predictions for video {}/{}: {}".format(c+1,len(videos),video))
    
    # load features from disk
    features = np.load(get_features_path(video))

    dfs = []
    for i in range(sequence_length-1, len(features)):

        # get features for the clip
        features_frames = features[i-sequence_length+1:i+1,]
        features_frames = np.expand_dims(features_frames, axis=0)

        # run through model
        y_pred = architecture.model.predict(features_frames)

        # create pred dataframe
        df_pred = pd.DataFrame(y_pred[0]).T
        df_pred.columns = class_names
        df_pred.index = [i]
        dfs.append(df_pred)

    # join pred dataframe onto labels
    y_pred = pd.concat(dfs)
    y_pred['pred'] = y_pred.idxmax(axis=1)
    # align labels index
    y_labs = labels[labels['video']==video]
    y_labs.reset_index(inplace=True,drop=True)
    # join predictions on labels
    y_pred = pd.merge(y_pred, y_labs, left_index=True,right_index=True,how='left')
    y_pred['error'] = (y_pred['label'] != y_pred['pred']).astype(int)
    
    y_preds.append(y_pred)

preds_all = pd.concat(y_preds)

Computing frame predictions for video 1/46: s1-218
Computing frame predictions for video 2/46: s10-6558
Computing frame predictions for video 3/46: s11-7363
